# Load Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.optimize
import liblie as ll

%matplotlib qt


df_1 = pd.read_csv('data_calib/f-2-1-1-1-6-1-L_1.csv', header=None, delimiter=' ')
df_1.columns = ["time", "x", "y", "z"]
df_2 = pd.read_csv('data_calib/f-2-1-1-1-6-1-L_2.csv', header=None, delimiter=' ')
df_2.columns = ["time", "x", "y", "z"]
df_3 = pd.read_csv('data_calib/f-2-1-1-1-6-1-L_3.csv', header=None, delimiter=' ')
df_3.columns = ["time", "x", "y", "z"]

p1s_l = np.zeros((len(df_1), 4))
p2s_l = np.zeros((len(df_1), 4))
p3s_l = np.zeros((len(df_1), 4))

for ps, df in zip([p1s_l, p2s_l, p3s_l], [df_1, df_2, df_3]):
    for i in range(ps.shape[0]):
        ps[i] = np.array([df.iloc[i]['x'], 
                          df.iloc[i]['y'],
                          df.iloc[i]['z'],
                             1])
# p1s_l = p1s_l[:50]       
# p2s_l = p2s_l[:50]       
# p3s_l = p3s_l[:50]       
Dist_prism_12 = 0.3819811991689936
Dist_prism_13 = 0.4426382054042266
Dist_prism_23 = 0.2564685508415531

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot(p1s_l[:,0], p1s_l[:,2], p1s_l[:,3], color='tab:red')
ax.plot(p2s_l[:,0], p2s_l[:,2], p2s_l[:,3], color='tab:green')
ax.plot(p3s_l[:,0], p3s_l[:,2], p3s_l[:,3], color='tab:blue')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

Text(0.5, 0, 'z')

In [2]:
c1 = np.array([[  1.43887968,   3.75545863,  -0.57627539,   1.        ],
 [  0.48894384,   6.75926881,  -0.5869449 ,   1.        ],
 [  1.36502929,  12.08766919,  -0.54940165,   1.        ],
 [  2.08759776,  20.26933936,  -0.49206061,   1.        ],
 [  2.95924891,  29.45196985,  -0.40240111,   1.        ],
 [  4.96145448,  43.67246302,  -0.25069846,   1.        ],
 [ -2.74990705,   0.95319382,  -0.578599  ,   1.        ],
 [ -8.92016744,   2.69609736,  -0.56646922,   1.        ],
 [-17.46391439,   3.70710209,  -0.50579883,   1.        ],
 [-26.49787505,   3.71514406,  -0.45982965,   1.        ],
 [-34.55389862,   4.87882254,  -0.44752752,   1.        ],
 [-43.38713417,   5.76577935,  -0.42166513,   1.        ]])
c2 = np.array([[ -1.77937907,   2.64135816,  -0.63033238,   1.        ],
 [ -4.34148853,   4.47228736,  -0.63825173,   1.        ],
 [ -6.84448749,   9.25804343,  -0.6014729 ,   1.        ],
 [-11.18551417,  16.23081266,  -0.54386175,   1.        ],
 [-16.00845024,  24.0936953 ,  -0.4505375 ,   1.        ],
 [-22.95710727,  36.66284013,  -0.29647593,   1.        ],
 [ -3.43828009,  -2.11874632,  -0.63218799,   1.        ],
 [ -9.42188783,  -4.43221931,  -0.61884105,   1.        ],
 [-16.85698435,  -8.76120715,  -0.55458324,   1.        ],
 [-24.0818527 , -14.18423241,  -0.50882464,   1.        ],
 [-31.2175589 , -18.09612541,  -0.49414403,   1.        ],
 [-38.81076928, -22.69677762,  -0.46462055,   1.        ]])
c3 = np.array([[ -2.3052958,    0.63678683,  -0.49565694,   1.        ],
 [ -5.45438075,   0.61432696,  -0.50609356,   1.        ],
 [-10.28896689,   3.02399706,  -0.46898338,   1.        ],
 [-17.89260624,   6.12741005,  -0.41224264,   1.        ],
 [-26.41009228,   9.6693983 ,  -0.32270604,   1.        ],
 [-39.40973676,  15.77674074,  -0.17038278,   1.        ],
 [ -0.86149119,  -4.19376028,  -0.49941067,   1.        ],
 [ -4.35097891,  -9.576572  ,  -0.48687291,   1.        ],
 [ -7.83555428, -17.44310933,  -0.42465254,   1.        ],
 [-10.50796211, -26.07180827,  -0.38073216,   1.        ],
 [-13.9946294 , -33.42575968,  -0.36614078,   1.        ],
 [-17.44860434, -41.60421735,  -0.3403394 ,   1.        ]])

# Find transforms for both methods

In [3]:
def dist(xi1, xi2):
    return np.linalg.norm(xi1-xi2)
def dist2(a,b):
    return np.sum([(a_-b_)**2 for a_, b_ in zip(a,b)])
def plot_theodolite(ax, T, c1='tab:red', c2='tab:green', name=""):
    b = T@np.array([0,0,0,1])
    dx = T@np.array([1,0,0,1]) - b
    dy = T@np.array([0,1,0,1]) - b
    
    plt.arrow(*b[:2], *dx[:2], color=c1)
    plt.arrow(*b[:2], *dy[:2], color=c2)
    plt.text(*b[:2],name)

## old method

In [13]:
def cost_funb(c1, c2, c3, xi1, xi2):
    T12_ = ll.expm_se3(ll.wedge_se3(xi1))
    T13 = ll.expm_se3(ll.wedge_se3(xi2))
    
    c=[]
    c += [ dist(c1_, T12_@c2_) for c1_,c2_ in zip(c1, c2)]
    c += [ dist(c1_, T13@c3_) for c1_,c3_ in zip(c1, c3)]
    # c += [ dist2(T12@c2_, T13@c3_) for c2_,c3_ in zip(c2, c3)]
    
    return c

x0 = np.random.normal(size=12)
f = lambda x: cost_funb(c1, c2, c3, x[:6], x[6:])
res = scipy.optimize.least_squares(f, x0, 
    xtol=1e-10, gtol=1e-10, max_nfev=10000,
    method='lm', verbose=2)
nxi_12 = res.x[:6]
nxi_13 = res.x[6:]
print(res.message)

# print(xi1)
# print(xi2)

nT12 = ll.expm_se3(ll.wedge_se3(nxi_12))
nT13 = ll.expm_se3(ll.wedge_se3(nxi_13))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
# ax.set_xlim(-50,-20)
# ax.set_ylim(-15,15)
# ax.set_zlim(-15,15)

ax.scatter(c1[:,0], c1[:,1], c1[:,2], color='tab:red')

c21 = np.zeros(c2.shape)
for i in range(c2.shape[0]):
    c21[i] = nT12@c2[i]

ax.scatter(c21[:,0], c21[:,1], c21[:,2], color='tab:green')

c31 = np.zeros(c3.shape)
for i in range(c3.shape[0]):
    c31[i] = nT13@c3[i]

ax.scatter(c31[:,0], c31[:,1], c31[:,2], color='tab:blue')
# plt.xlim(-1,5)
# plt.ylim(0,10)

T12_gt = nT12
T13_gt = nT13

The maximum number of function evaluations is exceeded.
Function evaluations 10000, initial cost 9.8238e+03, final cost 5.5683e-05, first-order optimality 1.06e-03.
The maximum number of function evaluations is exceeded.


## new method

In [14]:
def cost_fun(p1s_l, p2s_l, p3s_l, xi_12, xi_13):
    T12 = ll.expm_se3(ll.wedge_se3(xi_12))
    T13 = ll.expm_se3(ll.wedge_se3(xi_13))
    c=[]
    c += [(dist(p1,T12@p2)-Dist_prism_12**2) for p1,p2 in zip(p1s_l, p2s_l)]
    c += [(dist(p1,T13@p3)-Dist_prism_13**2) for p1,p3 in zip(p1s_l, p3s_l)]
    c += [(dist(T12@p2,T13@p3)-Dist_prism_23**2) for p2,p3 in zip(p2s_l, p3s_l)]
    
    return c

x0 = [*nxi_12, *nxi_13]
f = lambda x: cost_fun(p1s_l, p2s_l, p3s_l, x[:6], x[6:])
res = scipy.optimize.least_squares(f, x0,
      # gtol=1e-10, xtol=1e-10, ftol=1e-10,
      method='lm',
      verbose=2)
# print(res)
print(res.success)
print(res.message)
# print(res.x)
xi_12 = res.x[:6]
xi_13 = res.x[6:]
T12 = ll.expm_se3(ll.wedge_se3(xi_12))
T13 = ll.expm_se3(ll.wedge_se3(xi_13))


plt.figure()
plt.axis('equal')
ax = plt.gca()

plot_theodolite(ax,np.eye(4,4), name="1")
plot_theodolite(ax,T12, name="2")
plot_theodolite(ax,T13, name="3")

`ftol` termination condition is satisfied.
Function evaluations 105, initial cost 1.8698e+02, final cost 2.0420e-02, first-order optimality 9.04e-04.
True
`ftol` termination condition is satisfied.


# Compare Results

## Align trajectories and look at errors

In [16]:
def cost_fun(xi):
    Tb = ll.expm_se3(ll.wedge_se3(xi))
    c = []
    c += [dist(p1, Tb@T12@p2) for p1,p2 in zip(c1,c2)]
    c += [dist(p1, Tb@T13@p3) for p1,p3 in zip(c1,c3)]
    c += [dist(Tb@T12@p2, Tb@T13@p3) for p2,p3 in zip(c2,c3)]
    
    return c

x0 = np.zeros((6))
res =  scipy.optimize.least_squares(cost_fun, x0, 
    xtol=1e-10, gtol=1e-10, ftol=1e-10, max_nfev=10000,
    method='lm', verbose=2)

print(res.x)

Tb = ll.expm_se3(ll.wedge_se3(res.x))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

p12 = np.zeros(c2.shape)
for i in range(p12.shape[0]):
    p12[i] = Tb@T12@c2[i]

ax.scatter(c1[:,0], c1[:,1], c1[:,2], color='tab:red')
ax.scatter(p12[:,0], p12[:,1], p12[:,2], color='tab:green', marker='d')
# plt.plot(p2s_l[:,0], p2s_l[:,1])

# p = df_2.iloc[-1]
# print(p)
# print(T12@p)
p13 = np.zeros(c3.shape)
for i in range(p13.shape[0]):
    p13[i] = Tb@T13@c3[i]
        
ax.scatter(p13[:,0], p13[:,1], p13[:,2], color='tab:blue', marker='X')

`ftol` termination condition is satisfied.
Function evaluations 80, initial cost 1.4196e+00, final cost 4.8892e-01, first-order optimality 3.32e-06.
[ 0.24497393 -0.10594986 -0.07951836  0.          0.          0.        ]


In [17]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_xlim(-25,-15)
ax.set_ylim(0,10)
ax.set_zlim(-5,5)

p12 = np.zeros(p2s_l.shape)
for i in range(p12.shape[0]):
    p12[i] = T12@p2s_l[i]

ax.plot(p1s_l[:,0], p1s_l[:,1], p1s_l[:,2], color='tab:red')
ax.plot(p12[:,0], p12[:,1], p12[:,2], color='tab:green')
# plt.plot(p2s_l[:,0], p2s_l[:,1])

# p = df_2.iloc[-1]
# print(p)
# print(T12@p)
p13 = np.zeros(p3s_l.shape)
for i in range(p13.shape[0]):
    p13[i] = T13@p3s_l[i]
        
plt.plot(p13[:,0], p13[:,1], p13[:,2], color='tab:blue')

#-------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------
p12 = np.zeros(p2s_l.shape)
for i in range(p12.shape[0]):
    p12[i] = nT12@p2s_l[i]

ax.plot(p1s_l[:,0], p1s_l[:,1], p1s_l[:,2], color='tab:brown', ls='--')
ax.plot(p12[:,0], p12[:,1], p12[:,2], color='tab:purple', ls='--')

p13 = np.zeros(p3s_l.shape)
for i in range(p13.shape[0]):
    p13[i] = nT13@p3s_l[i]
        
plt.plot(p13[:,0], p13[:,1], p13[:,2], color='tab:cyan', ls='--')

## Same but align on trajectory instead of control points

In [19]:
def cost_fun(xi):
    Tb = ll.expm_se3(ll.wedge_se3(xi))
    c = []
    c += [dist2(nT12@p, Tb@T12@p) for p in p2s_l]
    c += [dist2(nT13@p, Tb@T13@p) for p in p3s_l]
    
    return np.array(c)

x0 = np.zeros((6))
res =  scipy.optimize.least_squares(cost_fun, x0, 
    xtol=1e-15, gtol=1e-15, ftol=1e-15, max_nfev=10000,
    method='lm', verbose=2)

print(res.x)

Tb = ll.expm_se3(ll.wedge_se3(res.x))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_xlim(-25,-15)
ax.set_ylim(0,10)
ax.set_zlim(-5,5)

p12 = np.zeros(p2s_l.shape)
for i in range(p12.shape[0]):
    p12[i] = Tb@T12@p2s_l[i]

ax.plot(p1s_l[:,0], p1s_l[:,1], p1s_l[:,2], color='tab:red')
ax.plot(p12[:,0], p12[:,1], p12[:,2], color='tab:green')
# plt.plot(p2s_l[:,0], p2s_l[:,1])

# p = df_2.iloc[-1]
# print(p)
# print(T12@p)
p13 = np.zeros(p3s_l.shape)
for i in range(p13.shape[0]):
    p13[i] = Tb@T13@p3s_l[i]
        
plt.plot(p13[:,0], p13[:,1], p13[:,2], color='tab:blue')

#-------------------------------------------------------------------------------------

p12 = np.zeros(p2s_l.shape)
for i in range(p12.shape[0]):
    p12[i] = nT12@p2s_l[i]

ax.plot(p1s_l[:,0], p1s_l[:,1], p1s_l[:,2], color='tab:brown', ls='--')
ax.plot(p12[:,0], p12[:,1], p12[:,2], color='tab:purple', ls='--')
# plt.plot(p2s_l[:,0], p2s_l[:,1])

# p = df_2.iloc[-1]
# print(p)
# print(T12@p)
p13 = np.zeros(p3s_l.shape)
for i in range(p13.shape[0]):
    p13[i] = nT13@p3s_l[i]
        
plt.plot(p13[:,0], p13[:,1], p13[:,2], color='tab:cyan', ls='--')

plt.figure()
plt.boxplot(cost_fun(res.x))

`ftol` termination condition is satisfied.
Function evaluations 84, initial cost 2.1346e+01, final cost 3.8677e-01, first-order optimality 6.60e-07.
[ 0.24508238 -0.11228323 -0.0743031   0.          0.          0.        ]


{'whiskers': [<matplotlib.lines.Line2D at 0x7ff394300040>,
 'caps': [<matplotlib.lines.Line2D at 0x7ff3943002e0>,
 'boxes': [<matplotlib.lines.Line2D at 0x7ff3a8c15880>],
 'medians': [<matplotlib.lines.Line2D at 0x7ff394300f70>],
 'fliers': [<matplotlib.lines.Line2D at 0x7ff39431a580>],
 'means': []}